## 스타벅스 이벤트 관련 고객 설문 데이터

- 스타벅스 고객들의 이벤트 관련 설문에 응답한 데이터의 일부입니다.


- 해당 데이터에서 고객들이 이벤트에 대한 응답을 어떻게 하는지, 찾고 고객 프로모션 개선방안에 대한 인사이트를 찾습니다.

### 0. Data Description

1. Profile table

> profile 데이터는 설문에 참여한 스타벅스 회원에 관련된 정보가 담겨 있습니다.

> "Dimesional data about each person, including their age, salary, and gender. There is one unique customer for each record."


2. transcript

> 이벤트에 참여한 실제 유저들의 응답이 기록되어 있습니다.

> "Records show the different steps of promotional offers that a customer received. The different values of receiving a promotion are receiving, viewing, and completing. You also see the different transactions that a person made in the time since he became a customer. With all records, you see the day that they interacted with Starbucks and the amount that it is worth."


3. portfoilo

> 이벤트를 운영했던 내역에 관한 정보가 담겨 있습니다.

> "Information about the promotional offers that are possible to receive, and basic information about each one including the promotional type, duration of the promotion, reward, and how the promotion was distributed to customers."

### 1. 라이브러리 및 데이터 로드

- 분석에 필요한 데이터와, 라이브러리를 불러옵니다.

In [ ]:
# 데이터 분석 필수 라이브러리 4종 세트 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Starbucks Customer Data 폴더안에 있는 데이터 3개를 불러오기
base_path = '/content/drive/MyDrive/Colab Notebooks/data/starbucks-customer-data/'

transcript = pd.read_csv(base_path + 'transcript.csv').drop(columns=['Unnamed: 0'])
profile = pd.read_csv(base_path + 'profile.csv').drop('Unnamed: 0', axis=1)
portfolio = pd.read_csv(base_path + 'portfolio.csv').drop(columns=['Unnamed: 0'])

In [ ]:
profile.info()

### 2. 데이터 전처리

- 결측치가 존재하는 데이터를 찾아서, 결측치를 처리해줍니다.

In [ ]:
# 각 데이터에 결측치가 있는지 확인합니다.
profile.info()

In [ ]:
# 결측치를 포함하는 데이터들은 어떤 데이터들인지 확인합니다.
nulls = profile[profile.isnull().any(axis=1)]
#nulls.id.value_counts()
nulls.id.nunique()

In [ ]:
# 결측치를 처리해줍니다. # 평균과 같은 통계량으로 채워주거나, 버립니다.
profile = profile.dropna()
profile.info()

### 3. profile 데이터 분석

- 설문에 참여한 사람 중, 정상적인 데이터로 판단된 데이터에 대한 분석을 수행합니다.


- 각 column마다 원하는 통계량을 찾은 뒤, 해당 통계량을 멋지게 시각화해 줄 plot을 seaborn에서 가져와 구현합니다.

In [ ]:
profile

In [ ]:
# profile의 became_member_on 데이터를 시간 정보로 변환해줍니다.
profile.became_member_on = pd.to_datetime(profile.became_member_on.astype(str), format='%Y%m%d')

In [ ]:
profile.info()

#### 성별에 관한 분석

In [ ]:
profile.gender.value_counts()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=profile, x='gender', palette='Set2')
plt.show()

In [ ]:
pd.pivot_table(data=profile,index='gender', values='income')

#### 나이에 대한 분석

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=profile, x='age', bins=15, hue='gender',multiple='stack')
plt.show()

In [ ]:
pd.pivot_table(data=profile, index='gender', values=['age','income'])

#### 회원이 된 날짜에 대한 분석

In [ ]:
profile['join_year'] = profile.became_member_on.dt.year
profile['join_month'] = profile.became_member_on.dt.month
profile

In [ ]:
plt.figure(figsize=(8,6))
x = profile.join_year.value_counts().index
y = profile.join_year.value_counts().values
#sns.barplot(x=x, y=y, order=x)
sns.countplot(data=profile, x='join_year', palette='Set2')
plt.show()

In [ ]:
x = profile.join_month.value_counts().index
y = profile.join_month.value_counts().values

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=x, y=y, order=x)
#.countplot(data=profile, y='join_month')
plt.show()

#### 수입에 대한 분석

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=profile, y='income', palette='Set2', hue='gender',multiple='stack')
plt.show()

#### profile 데이터에 대한 상관관계 분석

In [ ]:
plt.figure(figsize=(6,6))
sns.heatmap(data=profile.corr(), square=True, annot=True)
plt.show()

### 4. transcript에 대한 분석

- 각 column마다 원하는 통계량을 찾은 뒤, 해당 통계량을 멋지게 시각화해 줄 plot을 seaborn에서 가져와 구현합니다.


- `person`과 `values` column은 분석 대상에서 제외합니다.


> values에 대한 패턴이 궁금하다면, 아래 코드를 참고해보세요 ! (배우지 않은 내용이 있어, 어려울 수 있습니다)

In [ ]:
transcript

#### event에 대한 분석

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=transcript, x='event', palette='Set2')
plt.show()

In [ ]:
pd.pivot_table(data=transcript, index='event', values='time')

#### time에 대한 분석

In [ ]:
temp = sorted(transcript.time.value_counts()[:6].index)
print(temp)
for i in range(len(temp)-1):
  print(temp[i+1] - temp[i], end=" ")

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=transcript, x='time', hue='event')
plt.show()

In [ ]:
temp_df = transcript.loc[transcript.time.isin(temp), :]
temp_df

In [ ]:
plt.figure(figsize=(8,6))
#sns.countplot(data=temp_df, x='event',palette='Set2', hue='time')
sns.countplot(data=temp_df, x='time',palette='Set2', hue='event')

plt.show()

#### (Advanced, optional) value에 대한 분석 

In [ ]:
# str로 되어있는 데이터를 dictionary로 변환
transcript.value = transcript.value.apply(lambda x:eval(x))
transcript

In [ ]:
list(transcript.value[0].keys())[0]

In [ ]:
# event에 따른 value의 차이 확인
transcript["value_key"] = transcript.value.apply(lambda x:list(x.keys())[0])
transcript["value_value"] = transcript.value.apply(lambda x:list(x.values())[0])
transcript

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=transcript, x="value_key")
plt.show()

In [ ]:
transcript.value_value.value_counts()

In [ ]:
temp = transcript.loc[transcript.value_key != "amount", "value_value"]
temp_df = pd.DataFrame({"id" : temp.values})
temp_df

In [ ]:
temp.value_counts()

In [ ]:
temp_portfolio = pd.merge(temp_df, portfolio, how="left", on="id")
temp_portfolio

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=temp_portfolio, x="channels", palette="Set2")
plt.show()